In [1]:
# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.

from dash import Dash, html, dcc
import plotly.express as px
import pandas as pd
from jupyter_dash import JupyterDash


In [4]:
import os

In [5]:
os.system("pwd")

/Users/tackyx/rust/finial_axum/dashweb


0

In [6]:
import os
all_klines_file_names = os.listdir("../kline_out/")
all_finial_file_names = os.listdir("../output_base/")

In [7]:
len(all_klines_file_names), len(all_finial_file_names)

(4025, 3933)

In [8]:
all_klines_file_names[0], all_finial_file_names[0]

('SH603105.csv', '600966.SH.csv')

In [9]:
# '600966.SH.csv' => 'SH603105.csv'
def get_kline_file_name(finial_file_name):
    sp = finial_file_name.split(".")
    return sp[1] + sp[0] + ".csv"

In [10]:
trans_file_names = [get_kline_file_name(f) for f in all_finial_file_names]

In [11]:
# 求交集 trans_file_names and all_klines_file_names
def get_intersection(list1, list2):
    return list(set(list1).intersection(set(list2)))


In [12]:
common_file_names = get_intersection(trans_file_names, all_klines_file_names)

In [13]:
len(common_file_names)

3933

In [14]:
common_file_names[0]

'SZ000806.csv'

In [15]:
len(all_finial_file_names), len(all_klines_file_names), len(common_file_names)

(3933, 4025, 3933)

In [47]:
import json
import httpx
import time
from concurrent.futures import ThreadPoolExecutor

def get_stock_info(code):
    print("get for ", code)
    r = httpx.get(f"https://www.zsxg.cn/api/v2/capital/info?code={code}&yearNum=6")
    data_res = json.loads(r.text)
    pdn = pd.DataFrame.from_dict(data_res["datas"]["comment_new"]["positive_new"])
    pdn["情绪"] = "积极"
    pdun = pd.DataFrame.from_dict(data_res["datas"]["comment_new"]['unpositive_new'])
    pdun["情绪"] = "消极"
    df = pd.concat([pdn, pdun], axis=0)
    df["简介"] = data_res["datas"]["briefing"]
    df["股票代码"] = code
    df["公司名称"] = data_res["datas"]["name"]
    df["三类行业"] = data_res["datas"]["industry3"]
    df["二类行业"] = data_res["datas"]["industry2"]
    df["一类行业"] = data_res["datas"]["industry"]
    df["区域"] = data_res["datas"]["area"]
    return df

def do_get_stock_info(code):
    try:
        time.sleep(0.1)
        return get_stock_info(code)
    except Exception as e:
        print("error for ", code, e)
        return None

get for  600526.SH


get for  601800.SH
get for  600527.SH
get for  601801.SH
get for  600528.SH
get for  601808.SH
get for  601811.SH
get for  601816.SH


In [48]:

# all code 
# read from fixtures/SH.code and fixtures/SZ.code
with open("../fixtures/SH.code", "r") as f:
    sh_code = f.read().splitlines()
with open("../fixtures/SZ.code", "r") as f:
    sz_code = f.read().splitlines()
# get all code
all_code = sh_code + sz_code
print("all code: {}".format(len(all_code)))
# cal data use function: cal_data,  
# with parrallel worker use concurrent.futures
st_time = time.time()
# dflist = []
# for code in all_code:
#     try:
#         df = get_stock_info(code)
#         dflist.append(df)
#     except Exception as e:
#         print(e)
#         print(code)
#         continue

with ThreadPoolExecutor(max_workers=10) as executor:
    dflist = executor.map(do_get_stock_info, all_code)
# concat all data filter None
df = pd.concat([d for d in dflist if d is not None], axis=0)
print("cost time: {}".format(time.time() - st_time))
df.to_csv("../fixtures/stock_info.csv", index=False)

get for  600529.SH
all code: 4025
get for  600004.SH
get for  600008.SH
get for  600000.SH
get for  600007.SH
get for  600015.SH
get for  600009.SH
get for  600010.SH
get for  600011.SH
get for  600012.SH
get for  600006.SH
get for  600530.SH
get for  600531.SH
get for  601818.SH
get for  601827.SH
get for  600532.SH
get for  600533.SH
get for  600535.SH
get for  601828.SH
get for  600016.SH
get for  600536.SH
get for  600017.SH
get for get for  600018.SH
 601838.SH
get for  600537.SH
get for  601857.SH
get for  600538.SH
get for  600019.SH
get for  600539.SH
get for  601858.SH
get for  600540.SH
get for  601860.SH
get for  601865.SH
get for  601866.SH
get for  600020.SH
get for  601869.SH
get for  600021.SH
get for  600543.SH
get for  600545.SH
get for  600022.SH
get for  600023.SH
get for  600025.SH
get for  601872.SH
get for  601877.SH
get for  600546.SH
get for  600547.SH
get for  600548.SH
get for  601878.SH
get for  601880.SH
get for  601881.SH
get for  600549.SH
get for  600550.

In [59]:
d1 = df.query("股票代码 == '600004.SH'")

In [60]:
d1.head()


,value,tag,情绪,简介,股票代码,公司名称,三类行业,二类行业,一类行业,区域
0,近5年，收现比达到 112% ，销售收入现金含量较强。,收现,积极,公司是国内三大门户复合型枢纽机场之一，是国家着力打造的珠三角世界级机场群及粤港澳大湾区的核心...,600004.SH,白云机场,机场,航空机场,交通运输,广州市
0,近3年营业收入每年下滑 16% ，最新季度归母净利润同比下滑 672% ，成长能力很弱。,成长,消极,公司是国内三大门户复合型枢纽机场之一，是国家着力打造的珠三角世界级机场群及粤港澳大湾区的核心...,600004.SH,白云机场,机场,航空机场,交通运输,广州市
1,毛利率为 -4.3% ，行业处于衰退期，或企业缺乏竞争力。,毛利,消极,公司是国内三大门户复合型枢纽机场之一，是国家着力打造的珠三角世界级机场群及粤港澳大湾区的核心...,600004.SH,白云机场,机场,航空机场,交通运输,广州市
2,最新综合估值高于近十年 91% 的时间，处于历史高位。,估值,消极,公司是国内三大门户复合型枢纽机场之一，是国家着力打造的珠三角世界级机场群及粤港澳大湾区的核心...,600004.SH,白云机场,机场,航空机场,交通运输,广州市
3,固定资产占总资产 68% ，为重资产模式，维持竞争优势成本较高。,资产,消极,公司是国内三大门户复合型枢纽机场之一，是国家着力打造的珠三角世界级机场群及粤港澳大湾区的核心...,600004.SH,白云机场,机场,航空机场,交通运输,广州市


In [54]:
# data_res = json.loads(r.text)

In [55]:
# data_res["datas"].keys()

In [56]:
# data_res["datas"]["comment_new"]

In [57]:
# pdn.head()

In [49]:
# pdun.head()


In [50]:
# pdn = pdn.append(pdun)

In [51]:
# pd.concat([pdn, pdun])

In [52]:
# print()

In [53]:
# print(data_res["datas"]["briefing"])

In [7]:
# if __name__ == '__main__':


SyntaxError: incomplete input (2487681370.py, line 1)

In [16]:
app.run_server(mode="inline")